In [ ]:
!pip install xgboost
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, RandomizedSearchCV, learning_curve
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, confusion_matrix, classification_report, accuracy_score, silhouette_score
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.manifold import TSNE, LocallyLinearEmbedding, MDS
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imPipeline
from statsmodels.stats.anova import anova_lm
from time import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
import statsmodels.api as sm
from statsmodels.formula.api import ols
from xgboost import XGBClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.pipeline import Pipeline
import plotly.express as px

# Section 1 - Classification

# 1.1. Preprocessing and EDA, Train and Test Data sets

Functions for data preprocessing, EDA, and Multicollinearity

In [ ]:
def load_data(filename):
    df = pd.read_csv(filename)
    return df

In [ ]:
def display_data_info(df):
    df.info()
    print(df.describe())

In [ ]:
# Detecting and counting empty cells, NaN, and Inf
def detect_and_count_issues(df):
    for column in df.columns:
        if df[column].dtype in [np.float64, np.float32, np.int64, np.int32]:
            # Only check for NaN and Inf in numeric columns
            nan_count = df[column].isna().sum()
            inf_count = np.isinf(df[column]).sum()
            empty_count = 0
        else:
            # For non-numeric columns, only checking for NaN and empty cells
            nan_count = df[column].isna().sum()
            inf_count = 0  # No Inf check for non-numeric data
            empty_count = (df[column] == '').sum()

        # Print the counts for the column
        print(f"Column: {column}")
        print(f"  NaN: {nan_count}")
        print(f"  Inf: {inf_count}")
        print(f"  Empty: {empty_count}\n")


In [ ]:
# Displaying duplicate values in the 'ID' column
def check_duplicates_in_id(df):
    # Check for duplicate entries in the 'ID' column
    duplicates = df[df.duplicated('ID')]
    num_duplicates = len(duplicates)
    print(f'Number of Duplicate Entries in ID: {num_duplicates}')


In [ ]:
# Imputation for numerical variables by entering median value into empty, NaN, and Inf cells
def impute_numerical_median(df, col):
    if col in df.columns:
        median = df[col].median()
        if median:
            print(f"median for '{col}' is: '{median}'")
            df[col] = df[col].fillna(median)
            # Optionally, handle Inf values if they are present
            df[col] = df[col].replace([np.inf, -np.inf], median)
        else:
            print(f"Column '{col}' not numerical.")
    else:
        print(f"Column '{col}' does not exist in the DataFrame.")
    return df

In [ ]:
# Imputing categorical variables using mode
def impute_categorical_mode(df, col):
    if col in df.columns:
        mode = df[col].mode()[0]
        if mode:
            df[col] = df[col].fillna(mode)
            df[col] = df[col].replace('', mode)
        else:
            print(f"Column '{col}' not categorical.")
    else:
        print(f"Column '{col}' does not exist in the DataFrame.")
    return df

In [ ]:
# Variable One-hot coding for categorical variables, this will applied to the 'Profession' variable
def transform_column(df, col):
    if col in df.columns:
        # One-hot encoding the specified column
        df = pd.get_dummies(df, columns=[col], prefix=col)

    else:
        print(f"Column '{col}' does not exist in the DataFrame.")
    return df

In [ ]:
# Check point function - Converting to numeric values
 # This was designed mainly to resolve an issue that can occur when One-hot coding results in True/False values instead of 0 and 1
def convert_to_numeric_columns(df):
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

In [ ]:
 # Check point function - Converting boolean columns to integers
 # This was designed mainly to resolve an issue that can occur when One-hot coding results in True/False values instead of 0 and 1
def bool_to_int(df):
    for col_bool in df.select_dtypes(include=[bool]).columns:
        df[col_bool] = df[col_bool].astype(int)

    return df

In [ ]:
# Check point function - Verifying all columns are numeric
def verify_numeric_columns(df):
    for column in df.columns:
        if not np.issubdtype(df[column].dtype, np.number):
            raise ValueError(f"Column '{column}' is not numeric after transformation.")


In [ ]:
# Visualizing distributions of numerical features
def visualize_numerical_features(df):
    numerical_features = ['Age', 'Work_Experience', 'Family_Size']
    n_cols = 3  # Number of plots per row
    n_rows = (len(numerical_features) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
    axes = axes.flatten()

    for i, feature in enumerate(numerical_features):
        sns.boxplot(x=df[feature], ax=axes[i])
        axes[i].set_title(f'Box Plot of {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('')

    for i in range(len(numerical_features), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizing the distribution of categorical features
def visualize_categorical_features(df):
    categorical_features = ['Gender','Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']
    n_cols = 3  # Number of plots per row
    n_rows = (len(categorical_features) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
    axes = axes.flatten()

    for i, feature in enumerate(categorical_features):
        sns.countplot(x=feature, data=df, ax=axes[i])
        axes[i].set_title(f'Distribution of {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Count')

    # Removing any empty subplots
    for i in range(len(categorical_features), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
# For visualizing customer segmentation for Catgorical and Numerical Variable

def plot_category(feature, df, figsize=None):
    A_count = df[df['Segmentation']=='A'].groupby([feature]).size()
    B_count = df[df['Segmentation']=='B'].groupby([feature]).size()
    C_count = df[df['Segmentation']=='C'].groupby([feature]).size()
    D_count = df[df['Segmentation']=='D'].groupby([feature]).size()
    labels = A_count.index

    x = np.arange(len(labels))  # the label locations
    width = 0.7  # the width of the bars

    if figsize:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig, ax = plt.subplots()

    total_counts = df.groupby([feature]).size()

    rects1 = ax.bar(x - width/3, round(A_count * 100 / total_counts, 2), width/5, label='A')
    rects2 = ax.bar(x - width/8, round(B_count * 100 / total_counts, 2), width/5, label='B')
    rects3 = ax.bar(x + width/8, round(C_count * 100 / total_counts, 2), width/5, label='C')
    rects4 = ax.bar(x + width/3, round(D_count * 100 / total_counts, 2), width/5, label='D')

    ax.set_ylabel('Count')
    ax.set_title(f'Based on {feature}')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=80)
    ax.legend(loc=0, bbox_to_anchor=(1, 1))

    ax.bar_label(rects1, padding=1)
    ax.bar_label(rects2, padding=1)
    ax.bar_label(rects3, padding=1)
    ax.bar_label(rects4, padding=1)

    fig.tight_layout()
    plt.show()

def plot_numerical(feature, df, figsize=None):
    fig = plt.figure(figsize=(10, 6))

    sns.kdeplot(df[df['Segmentation']=='A'][feature], label='Segmentation A')
    sns.kdeplot(df[df['Segmentation']=='B'][feature], label='Segmentation B')
    sns.kdeplot(df[df['Segmentation']=='C'][feature], label='Segmentation C')
    sns.kdeplot(df[df['Segmentation']=='D'][feature], label='Segmentation D')

    plt.title(f'Based on {feature}')
    plt.legend()
    plt.show()

def plot_pie(feature, df):
    plot_data = df.groupby([feature, 'Segmentation']).size().reset_index(name='count')

    fig = px.sunburst(plot_data, path=[feature, 'Segmentation'], values='count', color=feature,
                      title=f'Affect of {feature} on Customer Segmentation', width=400, height=400)

    fig.update_layout(plot_bgcolor='white', title_font_family='Calibri Black', title_font_color='#092e5b',
                      title_font_size=20, title_x=0.5)

    fig.update_traces(textinfo='label+percent parent')
    fig.show()

In [ ]:
# Mapping Categorical Variables
def cat_to_num(df):

    # Transforming 'Gender' column
    df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

    # Transforming 'Ever_Married' column
    df['Ever_Married'] = df['Ever_Married'].map({'No': 0, 'Yes': 1})

    # Transforming 'Graduated' column
    df['Graduated'] = df['Graduated'].map({'No': 0, 'Yes': 1})

    # Transforming 'Spending_Score' column to numeric
    df['Spending_Score'] = df['Spending_Score'].map({'Low': 1, 'Average': 2, 'High': 3})

    # Converting boolean values to integers (0s and 1s)
    for col in df.select_dtypes(include=[bool]).columns:
        df[col] = df[col].astype(int)

    # If 'Var_1' is a categorical variable, convert it to numeric
    df['Var_1'] = df['Var_1'].astype('category').cat.codes

    # Ensuring 'Segmentation' is converted to numeric
    df['Segmentation'] = df['Segmentation'].astype('category').cat.codes

    # Verifying the new columns
    print('After transformation:')
    print(df.head())

    # Printing the data types of each column
    print("\nData types of each column:")
    print(df.dtypes)

    return df

In [ ]:
# Handling outliers by IQR
# This is to assigned to specific numerical variables that may contain outliers
def handle_outliers_iqr(df):
    # Specifying the numeric columns to check for outliers
    numerical_features = ['Age', 'Work_Experience', 'Family_Size']

    outliers_info = {}
    for feature in numerical_features:
        if feature in df.columns:
            Q1 = df[feature].quantile(0.25)
            Q3 = df[feature].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Counting the number of outliers
            num_outliers = ((df[feature] < lower_bound) | (df[feature] > upper_bound)).sum()
            outliers_info[feature] = num_outliers

            # Removing the outliers
            df = df[~((df[feature] < lower_bound) | (df[feature] > upper_bound))]

    return df, outliers_info

In [ ]:
# Correlation Matrix Heatmap
def compute_correlation_matrix(df):
    plt.figure(figsize=(20, 15))
    correlation_matrix = df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation Matrix')
    plt.show()

In [ ]:
# ANOVA Analysis
def run_anova(df):
    categorical_vars_corrected = ['Gender','Ever_Married', 'Graduated', 'Spending_Score', 'Var_1', 'Profession_Artist', 'Profession_Doctor', 'Profession_Engineer', 'Profession_Entertainment',
                                  'Profession_Executive', 'Profession_Healthcare', 'Profession_Homemaker', 'Profession_Lawyer', 'Profession_Marketing']

    anova_results = {}
    for var in categorical_vars_corrected:
        model_anova = ols(f'Segmentation ~ C({var})', data=df).fit()
        anova_table = anova_lm(model_anova, typ=2)
        anova_results[var] = anova_table

    print("ANOVA Analysis Results:")
    for var, results in anova_results.items():
        print(f"\nANOVA for {var}:\n", results)

In [ ]:
# Calculating VIF for detecting multicollinearity
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = df.columns
    vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    print("Variance Inflation Factor (VIF):")
    print(vif_data)

In [ ]:
# Check point - function - Any Missing Values still?
def missing_values_after_imputation(df):
    missing_values_after_imputation = df.isnull().sum()
    print("Missing values after imputation:\n", missing_values_after_imputation)

    return df

In [ ]:
# Removing 'ID' column
def remove_id(df):
    # Drop the 'ID' column
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])
    else:
        print("ID column not found.")
    return df

Combining functions to prepare datasets for analysis and visualization

In [ ]:
''' This is a global function designed to handle all initial preparing of both datasets:
- loading the datasets
- Displaying number of empty, NaN and Inf values in each column
- Detecting duplicates in 'ID'
- Imputing numerical variables by median and categorical variables by mode
- Visualizing distributions of numerical features
- Visualizing the distribution of categorical features
- One-Hot coding, to applied to 'Profession'
- Converting to numeric values
- Converting boolean columns to integers
- Verifying all columns are numeric
- Final validation of data integrity '''

def datasets_1(filename):
    df = load_data(filename)
    display_data_info(df)
    detect_and_count_issues(df)
    check_duplicates_in_id(df)
    df = impute_numerical_median(df, 'Age')
    df = impute_numerical_median(df, 'Work_Experience')
    df = impute_numerical_median(df, 'Family_Size')
    df = impute_categorical_mode(df, 'Ever_Married')
    df = impute_categorical_mode(df, 'Graduated')
    df = impute_categorical_mode(df, 'Profession')
    df = impute_categorical_mode(df, 'Var_1')
    detect_and_count_issues(df)
    display_data_info(df)
    missing_values_after_imputation(df)

    return df

# EDA
def datasets_2(df):
    visualize_numerical_features(df)
    visualize_categorical_features(df)

def customersegment1(df):
    for feature in ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']:
        plot_pie(feature, df)

def customersegment2(df):
    for feature in ['Profession', 'Var_1']:
        plot_pie(feature, df)

def customersegment3(df):
    for feature in ['Age', 'Work_Experience', 'Family_Size']:
        plot_numerical(feature, df)


# This one converts categorical to numeric, One-hot coding for 'Profession', verifying data integrity
def datasets_3(df):
    df = cat_to_num(df)
    df = transform_column(df, 'Profession')
    df = convert_to_numeric_columns(df)
    df = bool_to_int(df)
    verify_numeric_columns(df)
    detect_and_count_issues(df)

    return df


# This one is for visualizations after converting all values to numeric, here for correlation matrix
def analyses(df):
    compute_correlation_matrix(df)

# Analyzing multicollinearity
def multicollinearity(df):
    run_anova(df)
    calculate_vif(df)

In [ ]:
# Loading and defining dataframes for train and test, detecting nulls, NaN, Inf, imputation
df_train = datasets_1('Train.csv')
df_test = datasets_1('Test.csv')

In [ ]:
display_data_info(df_train)

In [ ]:
# Box plots and bar plots for both datasets
datasets_2(df_train)
datasets_2(df_test)

In [ ]:
# Converting categorical to numeric, One-hot coding for 'Profession', verifying data integrity
df_train = datasets_3(df_train)
df_test = datasets_3(df_test)

In [ ]:
# Data validation
display_data_info(df_train)
display_data_info(df_test)

In [ ]:
# Handling outliers, Train
df_train, outliers_info = handle_outliers_iqr(df_train)
print("Number of outliers detected and removed per column:")
print(outliers_info)

In [ ]:
display_data_info(df_train)

In [ ]:
# Handling outliers, Test
df_test, outliers_info = handle_outliers_iqr(df_test)
print("Number of outliers detected and removed per column:")
print(outliers_info)

In [ ]:
display_data_info(df_test)

In [ ]:
# Correlation matrix heatmap
df_Matrix_train = df_train.copy()
df_Matrix_test = df_test.copy()

# Removing 'ID' column from the copied dataframes
if 'ID' in df_Matrix_train.columns:
    df_Matrix_train = df_Matrix_train.drop(columns=['ID'])
if 'ID' in df_Matrix_test.columns:
    df_Matrix_test = df_Matrix_test.drop(columns=['ID'])

# Running the analyses function on the copied dataframes
analyses(df_Matrix_train)
analyses(df_Matrix_test)

Detecting Multicollinearity


In [ ]:
# ANOVA and Multicollinearity by computing VIFs
multicollinearity(df_train)

In [ ]:
# ANOVA and Multicollinearity by computing VIFs
multicollinearity(df_test)

In [ ]:
# Identifying the smallest profession group
profession_columns = [col for col in df_train.columns if col.startswith('Profession_')]
smallest_group = df_train[profession_columns].sum().idxmin()
print(f'Smallest profession group: {smallest_group}')

# Removing the smallest profession group from both training and testing datasets
df_train = df_train.drop(columns=[smallest_group])
df_test = df_test.drop(columns=[smallest_group])

print(f"Dropped column: {smallest_group}")

# Recalculate VIF
calculate_vif(df_train.drop(columns=['Segmentation']))

In [ ]:
# VIF >= 10 as cutoff for multicollinearirty?
# Thus, due to high VIF, we will remove 'Age' from the datasets
df_train = df_train.drop(columns=['Age'])
df_test = df_test.drop(columns=['Age'])

print("Dropped column: Age")

# Recalculating VIF
calculate_vif(df_train.drop(columns=['Segmentation']))

In [ ]:
# Removing 'ID' Column from both datasets
df_train = remove_id(df_train)
df_test = remove_id(df_test)

In [ ]:
# Recalculating VIF
calculate_vif(df_train.drop(columns=['Segmentation']))

At this point, both datasets have been preprocessed and are ready for classification models!

# 1.2. Preparing Datasets for Modeling

Defining Train and Test

In [ ]:
# Defining train and target for modeling, with tarin transformed dataset (df_train) as training, test transformed dataset (df_test) as test
X_train = df_train.drop(columns=['Segmentation'])
y_train = df_train['Segmentation']

X_test = df_test.drop(columns=['Segmentation'])
y_test = df_test['Segmentation']


Functions for Confusion Matrices and Learning Curves

In [ ]:
# For modeling - Functions for confusion matrices, learning curves
def plot_confusion_matrix(cm, title, save_path=None):
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy'):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    # Calculating learning curve
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring)

    # Calculating mean and standard deviation for training and test scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plotting learning curve
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")

    plt.show()
    # return plt

Feature Selection by Ridge and Lasso Regression

In [ ]:
# Lasso Regression for feature selection

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
lasso_pred = lasso.predict(X_test_scaled)
lasso_mse = mean_squared_error(y_test, lasso_pred)
lasso_r2 = r2_score(y_test, lasso_pred)
print(f"Lasso Regression MSE: {lasso_mse}")
print(f"Lasso Regression R^2: {lasso_r2}")

# Feature Importance from Lasso Regression
lasso_coefficients = pd.Series(lasso.coef_, index=X_train.columns)
lasso_coefficients = lasso_coefficients[lasso_coefficients != 0]
print(f"Lasso selected {len(lasso_coefficients)} features from {X_train.shape[1]}")
print("Selected features and their coefficients:")
print(lasso_coefficients)

# Plotting Lasso Feature Importance (absolute values to avoid negative importance)
plt.figure(figsize=(10, 6))
lasso_coefficients.abs().sort_values().plot(kind='barh', color='blue')
plt.title('Feature Importance from Lasso Regression')
plt.xlabel('Absolute Coefficient Value')
plt.ylabel('Features')
plt.show()

# Further Evaluation with Cross-Validation
lasso_cv_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Lasso Regression CV MSE: {-np.mean(lasso_cv_scores)}")

# 1.3. Classification models

PCA Dimensionality Reduction

Classifiers and Hyperparameter tuning

Ensemble Models

SVM

Class Imbalance

Feature Importance

Learning Curves

Evaluating all models

Pipeline with PCA and DecisionTree Classifier

In [ ]:
# Creating a pipeline with PCA for dimensionality reduction, DecisionTree Classifer
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Training and evaluating the pipeline
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred)}")
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")
plot_confusion_matrix(cm, "Confusion Matrix for Decision Tree")

GridSearchCV Hyperparameter Tuning

In [ ]:
# Hyperparameters tuning with GridSearchCV
# Defining the parameter grid
param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10]
}

# Creating the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fitting the model and finding the best parameters
grid_search.fit(X_train, y_train)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-validation Accuracy: {best_score}")

# Evaluating on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
plot_confusion_matrix(cm, "Confusion Matrix for GridSearchCV Best Model")

RandomizedSearchCV Hyperparameter Tuning

In [ ]:
# Hyperparameters tuning with RandomizedSearchCV
# Defining the parameter grid
param_dist = {
    'classifier__max_depth': [5, 10, 15, 20],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 5, 10]
}
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

# Best parameters
best_params = random_search.best_params_
best_score = random_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-validation Accuracy: {best_score}")

# Evaluating on test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
plot_confusion_matrix(cm, "Confusion Matrix for RandomizedSearchCV Best Model")

Ensemble Models

In [ ]:
# Ensemble models
# Defining the ensemble models
ensemble_models = {
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Gradient Boosted Trees': GradientBoostingClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Training and evaluating models
for model_name, model in ensemble_models.items():
    # No PCA in the pipeline in order to plot feature importance correctly
    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:\n", report)
    print("\n" + "="*60 + "\n")

    # Plotting feature importance for ensemble models
    if hasattr(model, 'feature_importances_'):
        feature_importances = model.feature_importances_

        # Sorting feature importances in descending order
        indices = np.argsort(feature_importances)[::-1]

        # Rearranging feature names so they match the sorted feature importances
        names = [X_train.columns[i] for i in indices]

        plt.figure(figsize=(10, 6))
        plt.title(f"Feature Importance for {model_name}")
        plt.bar(range(X_train.shape[1]), feature_importances[indices])
        plt.xticks(range(X_train.shape[1]), names, rotation=90)
        plt.xlabel('Features')
        plt.ylabel('Importance Score')
        plt.show()

        # Display the importance scores
        print("Feature importances:")
        for name, importance in zip(names, feature_importances[indices]):
            print(f"{name}: {importance:.4f}")

Learning Curves for AdaBoost and Gradient Boosted Trees

In [ ]:
# Plotting learning curves for AdaBoost and Gradient Boosted Trees
plot_learning_curve(AdaBoostClassifier(random_state=42), "Learning Curve (AdaBoost)", X_train, y_train, cv=5, n_jobs=-1)
plot_learning_curve(GradientBoostingClassifier(random_state=42), "Learning Curve (Gradient Boosted Trees)", X_train, y_train, cv=5, n_jobs=-1)


Support Vector Machine (SVM) Model

In [ ]:
# Defining the SVM model parameters
svm = SVC(kernel='linear', random_state=42, probability=True)

# Creating a pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
    ('classifier', svm)
])

# Training and evaluating the SVM model
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model: Support Vector Machine (SVM)")
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)
print("\n" + "="*60 + "\n")

Treating Class Imbalance

In [ ]:
# Addressing possible class imbalance
# Creating pipeline with SMOTE
pipeline = imPipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
    ('smote', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Training the model
pipeline.fit(X_train, y_train)

# Predicting and evaluating
y_pred = pipeline.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
plot_confusion_matrix(confusion_matrix(y_test, y_pred), "Confusion Matrix with SMOTE")


# Creating pipeline with RandomUnderSampler
undersample_pipeline = imPipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
    ('undersample', RandomUnderSampler(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Training the model with RandomUnderSampler
undersample_pipeline.fit(X_train, y_train)

# Predicting and evaluating with RandomUnderSampler
y_pred_undersample = undersample_pipeline.predict(X_test)
print("Confusion Matrix with RandomUnderSampler:")
print(confusion_matrix(y_test, y_pred_undersample))
print("\nClassification Report with RandomUnderSampler:\n", classification_report(y_test, y_pred_undersample))
plot_confusion_matrix(confusion_matrix(y_test, y_pred_undersample), "Confusion Matrix with RandomUnderSampler")


# Using class weights in the DecisionTreeClassifier
class_weight_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
    ('classifier', DecisionTreeClassifier(class_weight='balanced', random_state=42))
])

# Training the model with class weights
class_weight_pipeline.fit(X_train, y_train)

# Predicting and evaluating with class weights
y_pred_class_weight = class_weight_pipeline.predict(X_test)
print("Confusion Matrix with Class Weights:")
print(confusion_matrix(y_test, y_pred_class_weight))
print("\nClassification Report with Class Weights:\n", classification_report(y_test, y_pred_class_weight))
plot_confusion_matrix(confusion_matrix(y_test, y_pred_class_weight), "Confusion Matrix with Class Weights")

Feature Importance

In [ ]:
# Feature Importance with Permutation Importance
# Fitting a model directly on scaled data
rf_original = RandomForestClassifier(random_state=42)
rf_original.fit(X_train_scaled, y_train)  # X_train_scaled is from original features

# Assessing permutation importance
results = permutation_importance(rf_original, X_train_scaled, y_train, n_repeats=10, random_state=42, n_jobs=-1)

# Plotting the results
sorted_idx = results.importances_mean.argsort()
plt.figure(figsize=(10, 8))
plt.boxplot(results.importances[sorted_idx].T, vert=False, labels=X_train.columns[sorted_idx])
plt.title("Permutation Feature Importance")
plt.show()

# 1.4. Summarizing and Comparing Models

In [ ]:
# Final evaluation of all models with the best parameters and techniques (class imbalance excluded)

# Best parameters found from GridSearchCV and RandomizedSearchCV
best_params_grid_search = {'max_depth': 5, 'min_samples_split': 2}
best_params_random_search = {'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 2}

# List of final models to be evaluated
final_models = {
    'Decision Tree with GridSearchCV': DecisionTreeClassifier(random_state=42, **best_params_grid_search),
    'Decision Tree with RandomizedSearchCV': DecisionTreeClassifier(random_state=42, **best_params_random_search),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosted Trees': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'SVM': SVC(kernel='linear', random_state=42, probability=True)
}

# Dictionary to store model performance
model_performance = {}

# Evaluate each model
for model_name, model in final_models.items():
    # No PCA in the pipeline for feature importance
    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])

    # Performing cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    mean_cv_score = np.mean(cv_scores)
    std_cv_score = np.std(cv_scores)

    # Fitting the pipeline on the full training data
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    # Storing model performance
    model_performance[model_name] = {
        'pipeline': pipeline,
        'cv_score_mean': mean_cv_score,
        'cv_score_std': std_cv_score,
        'test_accuracy': accuracy,
        'confusion_matrix': cm,
        'classification_report': report
    }

    print(f"Model: {model_name}")
    print(f"Cross-Validation Accuracy: {mean_cv_score:.4f} ± {std_cv_score:.4f}")
    print(f"Test Accuracy: {accuracy}")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:\n", report)
    print("\n" + "="*60 + "\n")

# Finding the best-performing models
sorted_models = sorted(model_performance.items(), key=lambda x: x[1]['test_accuracy'], reverse=True)
best_models = sorted_models[:2]  # Choose the top 2 models

# Plotting feature importance for the best-performing models and learning curves
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

for i, (model_name, performance) in enumerate(best_models):
    if hasattr(performance['pipeline'].named_steps['classifier'], 'feature_importances_'):
        feature_importances = performance['pipeline'].named_steps['classifier'].feature_importances_

        # Sorting feature importances in descending order
        indices = np.argsort(feature_importances)[::-1]

        # Rearranging feature names so they match the sorted feature importances
        names = [X_train.columns[i] for i in indices]

        ax = axes[i, 0]
        ax.bar(range(X_train.shape[1]), feature_importances[indices])
        ax.set_title(f"Feature Importance for {model_name}")
        ax.set_xticks(range(X_train.shape[1]))
        ax.set_xticklabels(names, rotation=90)
        ax.set_xlabel('Features')
        ax.set_ylabel('Importance Score')

        # Displaying the importance scores
        print(f"Feature importances for {model_name}:")
        for name, importance in zip(names, feature_importances[indices]):
            print(f"{name}: {importance:.4f}")

    # Plotting learning curve for the best-performing models
    train_sizes, train_scores, test_scores = learning_curve(
        performance['pipeline'], X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 5), scoring='accuracy'
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    ax = axes[i, 1]
    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std, alpha=0.1, color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.1, color="g")
    ax.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    ax.set_title(f"Learning Curve for {model_name}")
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    ax.legend(loc="best")

plt.tight_layout()
plt.show()

In [ ]:
# Plotting learning curves for final models
plot_learning_curve(final_models['AdaBoost'], "Learning Curve (AdaBoost)", X_train, y_train, cv=5, n_jobs=-1)
plot_learning_curve(final_models['Gradient Boosted Trees'], "Learning Curve (Gradient Boosted Trees)", X_train, y_train, cv=5, n_jobs=-1)

#plt.show()

In [ ]:
# Evaluating each model and find the best one based on test accuracy and cross-validation score
best_model_name = None
best_model = None
best_test_accuracy = 0
best_cv_score_mean = 0

for model_name, model in final_models.items():
    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=0.95)),  # Retain 95% of the variance
        ('classifier', model)
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    mean_cv_score = np.mean(cv_scores)
    std_cv_score = np.std(cv_scores)

    # Fit the pipeline on the full training data
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    if test_accuracy > best_test_accuracy and mean_cv_score > best_cv_score_mean:
        best_test_accuracy = test_accuracy
        best_cv_score_mean = mean_cv_score
        best_model_name = model_name
        best_model = pipeline

    print(f"Model: {model_name}")
    print(f"Cross-Validation Accuracy: {mean_cv_score:.4f} ± {std_cv_score:.4f}")
    print(f"Test Accuracy: {test_accuracy}")
    print("\n" + "="*60 + "\n")

# Printing the best model
print(f"Best Model: {best_model_name}")
print(f"Best Model Test Accuracy: {best_test_accuracy}")
print(f"Best Model Cross-Validation Accuracy: {best_cv_score_mean:.4f}")

# 1.5. t-SNE for classification visualization

In [ ]:
# t-SNE Visualization for the Best Performing Model

# Assuming best_model is obtained from RandomizedSearchCV
best_model = random_search.best_estimator_

# Fitting the best model pipeline to the training data again if needed
best_model.fit(X_train, y_train)

# Getting the outputs of the PCA step
pca = best_model.named_steps['pca']
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Concatenating the PCA-transformed train and test data
X_combined_pca = np.vstack((X_train_pca, X_test_pca))
y_combined = np.concatenate((y_train, y_test))

# Applying t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_combined_pca)

# Creating a DataFrame for the t-SNE result
tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
tsne_df['Segmentation'] = y_combined

# Plotting the t-SNE result
plt.figure(figsize=(10, 8))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='Segmentation', data=tsne_df, palette='viridis', alpha=0.7)
plt.title('t-SNE Visualization of Segmentation')
plt.show()

# Section 2 - Alternative Segmentation 
Analysis of Train alone, Train Clustering, Test Clustering, Analysis of Alternative Classification

# 2.1. Train Dataset Model Evaluation

In [ ]:
# Splitting the dataset
X = df_train.drop(columns=['Segmentation'])
y = df_train['Segmentation']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# List of classification models to evaluate
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'ExtraTrees': ExtraTreesClassifier(random_state=42),
    'Bagging': BaggingClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(kernel='linear', random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(random_state=42)
}

In [ ]:
# Model Evaluation
def evaluate_model(model, X_train, y_train, X_val, y_val):
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        cm = confusion_matrix(y_val, y_pred)
        cr = classification_report(y_val, y_pred)
        return accuracy, cm, cr
    except Exception as e:
        print(f"Error evaluating model {model}: {e}")
        return None, None, None

# Evaluate each model and print results
results = {}
for name, model in models.items():
    accuracy, cm, cr = evaluate_model(model, X_train_scaled, y_train, X_val_scaled, y_val)
    if accuracy is not None:
        results[name] = {
            'accuracy': accuracy,
            'confusion_matrix': cm,
            'classification_report': cr
        }

In [ ]:
# Finding the best model
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model_accuracy = results[best_model_name]['accuracy']
best_model_cm = results[best_model_name]['confusion_matrix']

print(f'Best Model: {best_model_name}')
print(f'Accuracy: {best_model_accuracy}')
print(f'Confusion Matrix:\n{best_model_cm}')

In [ ]:
# Hyperparameter tuning for improving prediction model performance
# Defining a parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 150],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 4],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initializing the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Initializing GridSearchCV
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fitting GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Getting the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Evaluating the best model on the validation set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
cr = classification_report(y_val, y_pred)

print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score}')
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{cm}')
print(f'Classification Report:\n{cr}')

# 2.2. t-SNE Visualization of the Train Data

In [ ]:
# t-SNE for Data Visualization
# Converting scaled data back to DataFrame for easier handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_train_scaled_df['Segmentation'] = y_train.values

# Applying t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_reduced = tsne.fit_transform(X_train_scaled)

# Plotting the t-SNE output
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y_train, cmap='viridis', alpha=0.6)
plt.title('t-SNE visualization of the Data')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(scatter)
plt.show()

plt.figure(figsize=(10,8))
sns.scatterplot(x=X_reduced[:, 0], y=X_reduced[:, 1], hue='Segmentation',
               palette=sns.color_palette('hls', len(y_train.unique())),
               data=X_train_scaled_df,
               alpha=0.6)
plt.title('t-SNE visualization with Hue by Original Segmentation')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# 2.3. Train Dataset Clustering

In [ ]:
# Dropping the target variable 'Segmentation' and create a new DataFrame for clustering
df_clustering = df_train.drop(columns=['Segmentation'])

In [ ]:
# Standardizing the data
scaler = StandardScaler()
X_clustering_scaled = scaler.fit_transform(df_clustering)

In [ ]:
# Clustering using K-means and Agglomerative Clustering
# Applying K-Means Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(X_clustering_scaled)

# Applying Agglomerative Clustering
agg_clustering = AgglomerativeClustering(n_clusters=4)
agg_labels = agg_clustering.fit_predict(X_clustering_scaled)

In [ ]:
# t-SNE visualization of K-Means clusters
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_clustering_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=kmeans_labels, cmap='viridis', alpha=0.6)
plt.title('t-SNE visualization of K-Means Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(scatter)
plt.show()

# t-SNE visualization of Agglomerative Clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=agg_labels, cmap='viridis', alpha=0.6)
plt.title('t-SNE visualization of Agglomerative Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(scatter)
plt.show()

In [ ]:
# Dendrogram for Hierarchical Clustering
linked = linkage(X_clustering_scaled, 'ward')

plt.figure(figsize=(10, 7))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
plt.title('Dendrogram for Hierarchical Clustering')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

In [ ]:
# Silhouette Score for K-Means Clustering
kmeans_labels = kmeans.fit_predict(X_clustering_scaled)
kmeans_silhouette = silhouette_score(X_clustering_scaled, kmeans_labels)
print(f'Silhouette Score for K-Means: {kmeans_silhouette:.4f}')

In [ ]:
# Silhouette Score for Agglomerative Clustering
agg_labels = agg_clustering.fit_predict(X_clustering_scaled)
agg_silhouette = silhouette_score(X_clustering_scaled, agg_labels)
print(f'Silhouette Score for Agglomerative Clustering: {agg_silhouette:.4f}')

# 2.4. Train Dataset Model Evaluation with New Segmentation

In [ ]:
# Appending the cluster labels to the train dataframe
df_train['KMeans_Cluster'] = kmeans_labels
df_train['Agglomerative_Cluster'] = agg_labels

# Creating the 'Alt_Seg' column based on the clustering results using Agglomerative Clustering
df_train['Alt_Seg'] = agg_labels

print(df_train.head())

In [ ]:
# Removing 'Segmentation' and Clustering Columns
df_train = df_train.drop(columns=['KMeans_Cluster', 'Agglomerative_Cluster', 'Segmentation'])

# Defining the new features and target variable
X_alt = df_train.drop(columns=['Alt_Seg'])
y_alt = df_train['Alt_Seg']

# Splitting the data into training and validation sets
X_alt_train, X_alt_val, y_alt_train, y_alt_val = train_test_split(X_alt, y_alt, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_alt_train_scaled = scaler.fit_transform(X_alt_train)
X_alt_val_scaled = scaler.transform(X_alt_val)

# Training a model using RandomForestClassifier
alt_model = RandomForestClassifier(random_state=42)
alt_model.fit(X_alt_train_scaled, y_alt_train)

# Predicting the new segmentation on the validation set
y_alt_pred = alt_model.predict(X_alt_val_scaled)

# Evaluating the model
alt_accuracy = accuracy_score(y_alt_val, y_alt_pred)
alt_cm = confusion_matrix(y_alt_val, y_alt_pred)
alt_cr = classification_report(y_alt_val, y_alt_pred)

print(f'Accuracy of the model with Alt_Seg: {alt_accuracy}')
print(f'Confusion Matrix:\n{alt_cm}')
print(f'Classification Report:\n{alt_cr}')

In [ ]:
# Analyzing new classification
# Counting the number of unique classification values
num_classes = df_train['Alt_Seg'].nunique()
print(f'Number of unique classification values: {num_classes}')

# Counting the number of customers in each 'Alt_Seg' type
class_counts = df_train['Alt_Seg'].value_counts()
print(f'Number of customers in each Alt_Seg type:\n{class_counts}')

# Summary statistics for each 'Alt_Seg' type
class_summary = df_train.groupby('Alt_Seg').agg({
    'Work_Experience': ['mean', 'median', 'std'],
    'Family_Size': ['mean', 'median', 'std'],
    'Gender': lambda x: x.value_counts().idxmax(),
    'Ever_Married': lambda x: x.value_counts().idxmax(),
    'Graduated': lambda x: x.value_counts().idxmax(),
    'Spending_Score': lambda x: x.value_counts().idxmax(),
    'Var_1': lambda x: x.value_counts().idxmax(),
})

print(f'Summary statistics for each Alt_Seg type:\n{class_summary}')

In [ ]:
# Bar plot of customer distribution across segments
plt.figure(figsize=(10, 6))
df_train['Alt_Seg'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Customer Distribution Across Segments')
plt.xlabel('Segments')
plt.ylabel('Number of Customers')
plt.show()

In [ ]:
# Pie chart of customer distribution
plt.figure(figsize=(8, 8))
df_train['Alt_Seg'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette('pastel'))
plt.title('Customer Distribution Across Segments')
plt.ylabel('')
plt.show()

In [ ]:
# Box plots of numerical features across segments
numerical_features = ['Work_Experience', 'Family_Size']
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Alt_Seg', y=feature, data=df_train)
    plt.title(f'{feature} Distribution Across Segments')
    plt.xlabel('Segments')
    plt.ylabel(feature)
    plt.show()

In [ ]:
# Detailed profiles for each segment
for seg in df_train['Alt_Seg'].unique():
    segment_data = df_train[df_train['Alt_Seg'] == seg]
    print(f'\nProfile for Segment {seg}:')
    print(segment_data.describe(include='all'))

In [ ]:
# Feature importance
feature_importances = alt_model.feature_importances_

# Getting feature names
feature_names = X_alt.columns

# Creating a DataFrame for visualization
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Plotting feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df, palette='viridis')
plt.title('Feature Importance from RandomForestClassifier', fontsize=20, fontweight='bold')
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluating prediction models with new segmentation
# Splitting the dataset
X = df_train.drop(columns=['Alt_Seg'])
y = df_train['Alt_Seg']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# List of models to evaluate
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'ExtraTrees': ExtraTreesClassifier(random_state=42),
    'Bagging': BaggingClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(random_state=42)
}

def evaluate_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)
    cr = classification_report(y_val, y_pred)
    return accuracy, cm, cr

# Evaluating each model
results = {}
for name, model in models.items():
    accuracy, cm, cr = evaluate_model(model, X_train_scaled, y_train, X_val_scaled, y_val)
    results[name] = {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'classification_report': cr
    }

# Finding the best model
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model_accuracy = results[best_model_name]['accuracy']
best_model_cm = results[best_model_name]['confusion_matrix']

print(f'Best Model: {best_model_name}')
print(f'Accuracy: {best_model_accuracy}')
print(f'Confusion Matrix:\n{best_model_cm}')

# 2.5. Applying Clustering to the Test Dataset

In [ ]:
# Removing 'Segmentation' to apply new clustering on test_df
df_test = df_test.drop(columns=['Segmentation'])

In [ ]:
# Ensuring 'Test' dataset has the same columns as 'Train' dataset
missing_cols = set(X_train.columns) - set(df_test.columns)
for col in missing_cols:
    df_test[col] = 0  # Adding missing columns with default value 0

# Ensuring the same order of columns
df_test = df_test[X_train.columns]

print(df_test.head())

In [ ]:
# Standardizing the 'Test' data
X_test_scaled = scaler.transform(df_test)

# Predicting the 'Alt_Seg' on the 'Test' dataset using the best model
y_test_pred = best_model.predict(X_test_scaled)

# Creating a new dataframe with the predictions
df_test['Alt_Seg'] = y_test_pred

# Saving the predictions to a CSV file
df_test.to_csv('Test_with_Alt_Seg.csv', index=False)

print(df_test.head())

In [ ]:
# Count of each predicted segment
alt_seg_counts = df_test['Alt_Seg'].value_counts()
print("Count of each predicted Alt_Seg:")
print(alt_seg_counts)

# Convert counts to string for title
counts_str = ', '.join([f'Alt_Seg {seg}: {count}' for seg, count in alt_seg_counts.items()])

# Visualizing the distribution of the predicted labels
plt.figure(figsize=(10, 6))
sns.countplot(x='Alt_Seg', data=df_test, palette='viridis', hue='Alt_Seg', dodge=False, legend=False)
plt.title(f'Distribution of Predicted Alt_Seg in Test Dataset')
plt.xlabel('Alt_Seg')
plt.ylabel('Count')
plt.show()


In [ ]:
# Pie chart of customer distribution across segments in test data
plt.figure(figsize=(8, 8))
df_test['Alt_Seg'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette('pastel'))
plt.title('Customer Distribution Across Segments in Test Data')
plt.ylabel('')
plt.show()

In [ ]:
# Calculating silhouette score
silhouette_avg = silhouette_score(X_test_scaled, y_test_pred)
print(f'Silhouette Score for the clustering on test data: {silhouette_avg}')

In [ ]:
# Getting summary statistics
test_class_summary = df_test.groupby('Alt_Seg').agg({
    'Work_Experience': ['mean', 'median', 'std'],
    'Family_Size': ['mean', 'median', 'std'],
    'Gender': lambda x: x.value_counts().idxmax(),
    'Ever_Married': lambda x: x.value_counts().idxmax(),
    'Graduated': lambda x: x.value_counts().idxmax(),
    'Spending_Score': lambda x: x.value_counts().idxmax(),
    'Var_1': lambda x: x.value_counts().idxmax(),
})

print(f'Summary statistics for each Alt_Seg type in test data:\n{test_class_summary}')

In [ ]:
# Box plots of numerical features in test data, by segment
numerical_features = ['Work_Experience', 'Family_Size']
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Alt_Seg', y=feature, data=df_test)
    plt.title(f'{feature} Distribution Across Segments in Test Data')
    plt.xlabel('Segments')
    plt.ylabel(feature)
    plt.show()


In [ ]:
# t-SNE to the test data
tsne_test = TSNE(n_components=2, random_state=42)
X_tsne_test = tsne_test.fit_transform(X_test_scaled)

# t-SNE visualization of agglomerative clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_tsne_test[:, 0], X_tsne_test[:, 1], c=y_test_pred, cmap='viridis', alpha=0.6)
plt.title('t-SNE visualization of Test Data Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(scatter)
plt.show()

# 2.6. Evaluating Model on the Test Dataset with the New Segmentation

In [ ]:
# Defining df as train and test_df as test to apply models and see fit of the new classification on test
X_train_1 = df_train.drop(columns=['Alt_Seg'])
y_train_1 = df_train['Alt_Seg']

X_test_1 = df_test.drop(columns=['Alt_Seg'])
y_test_1 = df_test['Alt_Seg']

In [ ]:
# Standardizing the data
scaler_1 = StandardScaler()
X_train_scaled_1 = scaler_1.fit_transform(X_train_1)
X_test_scaled_1 = scaler_1.transform(X_test_1)

In [ ]:
# Training the model
model_1 = GradientBoostingClassifier(random_state=42)
model_1.fit(X_train_scaled_1, y_train_1)

# Predicting on the test set
y_test_pred_1 = model_1.predict(X_test_scaled_1)

# Evaluating the model
accuracy_1 = accuracy_score(y_test_1, y_test_pred_1)
cm_1 = confusion_matrix(y_test_1, y_test_pred_1)
cr_1 = classification_report(y_test_1, y_test_pred_1)

print(f'Accuracy of GradientBoostingClassifier on test data: {accuracy_1}')
print(f'Confusion Matrix:\n{cm_1}')
print(f'Classification Report:\n{cr_1}')

In [ ]:
# Summary statistics
test_class_summary_1 = df_test.groupby('Alt_Seg').agg({
    'Work_Experience': ['mean', 'median', 'std'],
    'Family_Size': ['mean', 'median', 'std'],
    'Gender': lambda x: x.value_counts().idxmax(),
    'Ever_Married': lambda x: x.value_counts().idxmax(),
    'Graduated': lambda x: x.value_counts().idxmax(),
    'Spending_Score': lambda x: x.value_counts().idxmax(),
    'Var_1': lambda x: x.value_counts().idxmax(),
})
print(f'Summary statistics for each Alt_Seg type in test data:\n{test_class_summary_1}')